# Piramide de vacinação brasileira

Fontes:

1. população brasileira - [Projeção da população do Brasil e das Unidades da Federação - IBGE](https://www.ibge.gov.br/apps/populacao/projecao//index.html)
2. população vacinada - [Registros de Vacinação COVID19 - OpenDataSUS](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8)

Baixe o ```.csv``` completo com os dados de vacinação no Brasil e substitua **adress** no código abaixo pelo endereço no computador onde o arquivo ```.csv``` está baixado.

In [ ]:
vac = pd.read_csv('adress',
                  error_bad_lines=False, nrows=1000000, delimiter = ';')

In [ ]:
vac.head()

No local de **date** coloque a data de corte para até onde você irá pegar dados. Se deseja pegar dados de vacinação até o dia 01/06/2021, por exemplo, coloque a data como 02/06/2021.

In [ ]:
dates = ['date'] #Substitua aqui a data de corte que deseja
dates_pd = pd.DataFrame({'dates': dates})
dates_pd['dates'] = pd.to_datetime(dates_pd['dates'], format = '%Y-%m-%d').dt.tz_localize(None)

rows_n = 2000000

pop_br = 212981000

man = [0.0356,0.0354,0.0357,0.0380,0.0411,0.0401,0.0403,0.0394,0.0358,0.0311,0.0285,0.0251,0.0205,
       0.0158,0.0113,0.0073,0.0045,0.0022,0.0013]
woman = [0.0340,0.0338,0.0342,0.0366,0.0402,0.0401,0.0410,0.0410,0.0379,0.0334,0.0311,0.0281,0.0238,
         0.0189,0.0142,0.0098,0.0067,0.0037,0.0026]

br = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                   'Male': np.array(man)*pop_br,
                   'Female': np.array(woman)*(-pop_br)})
AgeClass = ['90+','85-89','80-84','75-79','70-74','65-69','60-64','55-59','50-54','45-49','40-44','35-39','30-34','25-29','20-24','15-19','10-14','5-9','0-4']

for date in dates:
  dat = date
  print(f'Threshold date: {dat}')
  male_first = []
  female_first = []
  male_second = []
  female_second = []
  for i in range(32):
    print(f'Section number: {i+1}')
    if i == 0:
        vac = pd.read_csv('gdrive/MyDrive/COVID/Vaccination/Vacinacao_COVID_Brasil_completo.csv',
                        error_bad_lines=False, nrows=rows_n, delimiter = ';')
    elif i > 0:
      vac = pd.read_csv('gdrive/MyDrive/COVID/Vaccination/Vacinacao_COVID_Brasil_completo.csv',
                        error_bad_lines=False, skiprows=range(1, i*rows_n) , nrows=rows_n, delimiter = ';')
    vac['vacina_dataaplicacao'] = pd.to_datetime(vac['vacina_dataaplicacao'], format = '%Y-%m-%d').dt.tz_localize(None)
    vac = vac.loc[vac['vacina_dataaplicacao'] < dates_pd['dates'].values[0]]
    print(f"Last date in this section: {vac.sort_values('vacina_dataaplicacao').reset_index(drop=True)['vacina_dataaplicacao'][len(vac['vacina_dataaplicacao'])-1]}")
    vacina_1_dose = vac.loc[vac['vacina_descricao_dose'] == '\xa0\xa0\xa0\xa01ª\xa0Dose']
    vacina_2_dose = vac.loc[vac['vacina_descricao_dose'] == '\xa0\xa0\xa0\xa02ª\xa0Dose']
    for j in range(19):
      x = vacina_1_dose.loc[vacina_1_dose['paciente_idade'] >= j*5]
      x = x.loc[x['paciente_idade'] <= j*5 + 4]
      x2 = vacina_2_dose.loc[vacina_2_dose['paciente_idade'] >= j*5]
      x2 = x2.loc[x2['paciente_idade'] <= j*5 + 4]
      if j == 18:
        x = vacina_1_dose.loc[vacina_1_dose['paciente_idade'] >= j*5]
        x2 = vacina_2_dose.loc[vacina_2_dose['paciente_idade'] >= j*5]
      m = len(x.loc[x['paciente_enumsexobiologico'] == 'M'])
      m2 = len(x2.loc[x2['paciente_enumsexobiologico'] == 'M'])
      male_first.append(m)
      male_second.append(m2)
      f = len(x.loc[x['paciente_enumsexobiologico'] == 'F'])
      f2 = len(x2.loc[x2['paciente_enumsexobiologico'] == 'F'])
      female_first.append(-f)
      female_second.append(-f2)
  print(len(male_first))
  print(len(female_first))
  male_final_1 = np.zeros(19)
  female_final_1 =np.zeros(19)
  male_final_2 = np.zeros(19)
  female_final_2 =np.zeros(19)

  for i in range(int(len(male_first)/19)):
    male_final_1 += male_first[i*19:i*19+19]
    female_final_1 += female_first[i*19:i*19+19]
    male_final_2 += male_second[i*19:i*19+19]
    female_final_2 += female_second[i*19:i*19+19]

  print(male_final_1.shape)
  print(female_final_1.shape)

  df = pd.DataFrame({'Age': ['0-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90+'],
                   'Male_1dose': male_final_1,
                   'Female_1dose': female_final_1,
                   'Male_2dose': male_final_2,
                   'Female_2dose': female_final_2})
  print(df.head(5))
  #plt.figure(figsize=(10,8))
  bar_plot = sns.barplot(x='Male', y='Age', data=br, order=AgeClass, color = 'blue', alpha = 0.4, label = 'Homens população')
  bar_plot = sns.barplot(x='Female', y='Age', data=br, order=AgeClass, color = 'red', alpha = 0.4, label = 'Mulheres população')

  bar_plot = sns.barplot(x='Male_1dose', y='Age', data=df, order=AgeClass, color = 'blue', alpha = 0.7, label = 'Homens vacinados 1ª dose')
  bar_plot = sns.barplot(x='Female_1dose', y='Age', data=df, order=AgeClass, color = 'red', alpha = 0.7, label = 'Mulheres vacinadas 1ª dose')

  bar_plot = sns.barplot(x='Male_2dose', y='Age', data=df, order=AgeClass, color = 'darkblue', label = 'Homens vacinados 2ª dose')
  bar_plot = sns.barplot(x='Female_2dose', y='Age', data=df, order=AgeClass, color = 'darkred', label = 'Mulheres vacinadas 2ª dose')

  plt.ylabel('Grupo Etário', fontsize = 14)
  plt.xlabel('População (milhões)', fontsize = 14)
  plt.title(f'Pirâmide de vacinação ({dat})', fontsize = 18)
  plt.xticks([-10e6, -7.5e6, -5e6, -2.5e6, 0, 2.5e6, 5e6, 7.5e6, 10e6],
            labels = ['10','7.5','5','2.5','0','2.5','5','7.5','10'])
  plt.legend(title = 'Populações', bbox_to_anchor = (1.01,1), fontsize = 12)
  # plt.text(-10e6, 22, r'$\bf{Fontes:}$'
  # '\n'
  # '1. População brasileira: Projeção da população do Brasil e das Unidades da Federação - IBGE'
  # '\n'
  # '2. População vacinada: Registros de Vacinação COVID19 - OpenDataSUS', fontsize = 12)
  plt.savefig(f'vaccination_pyramid_{dat}.png', bbox_inches = 'tight')#, dpi = 300)
  plt.close()